# ⚡ Pinokio Lightning.ai - Permission Fix

**Fixed version for Lightning.ai platform**

Uses user-space directories to avoid permission errors.

---

In [ ]:
#@title ⚡ Lightning.ai Pinokio Setup
#@markdown Fixed for Lightning.ai permission restrictions

import os
import subprocess
import requests
import time
import json
from pathlib import Path

class PinokioLightning:
    def __init__(self):
        # Use home directory instead of system directories
        self.pinokio_path = os.path.expanduser("~/pinokio")
        self.binary_path = None
        self.server_process = None
        self.tunnel_process = None
        self.tunnel_url = None
        
    def setup_directory(self):
        print("📁 Setting up directory...")
        Path(self.pinokio_path).mkdir(parents=True, exist_ok=True)
        os.chdir(self.pinokio_path)
        print(f"✅ Created directory: {self.pinokio_path}")
        
    def download_pinokio(self):
        print("📥 Downloading Pinokio...")
        self.binary_path = os.path.join(self.pinokio_path, "Pinokio-linux.AppImage")
        
        if os.path.exists(self.binary_path):
            print("✅ Pinokio already downloaded")
            return True
            
        try:
            download_url = "https://github.com/pinokiocomputer/pinokio/releases/download/3.9.0/Pinokio-3.9.0.AppImage"
            result = subprocess.run(['wget', '-q', '--show-progress', '-O', self.binary_path, download_url], capture_output=True, text=True)
            
            if result.returncode != 0:
                print(f"❌ Download failed: {result.stderr}")
                return False
                
            os.chmod(self.binary_path, 0o755)
            
            if os.path.exists(self.binary_path) and os.access(self.binary_path, os.X_OK):
                print("✅ Pinokio downloaded and configured")
                return True
            else:
                print("❌ Binary not properly configured")
                return False
                
        except Exception as e:
            print(f"❌ Download error: {e}")
            return False
    
    def start_pinokio_server(self):
        print("⚡ Starting Pinokio on Lightning.ai...")
        
        if not self.binary_path or not os.path.exists(self.binary_path):
            print("❌ Pinokio binary not found")
            return False
            
        try:
            # Lightning.ai compatible environment
            env = os.environ.copy()
            env['PINOKIO_APP_PORT'] = '42000'
            
            # Use user home directory for everything
            home = os.path.expanduser("~")
            env['XDG_RUNTIME_DIR'] = os.path.join(home, '.xdg')
            env['XDG_DATA_HOME'] = os.path.join(home, '.local', 'share')
            env['XDG_CONFIG_HOME'] = os.path.join(home, '.config')
            env['XDG_CACHE_HOME'] = os.path.join(home, '.cache')
            
            # Create user directories
            for dir_path in [env['XDG_RUNTIME_DIR'], env['XDG_DATA_HOME'], env['XDG_CONFIG_HOME'], env['XDG_CACHE_HOME']]:
                Path(dir_path).mkdir(parents=True, exist_ok=True)
            
            # Disable D-Bus completely
            env['DBUS_SESSION_BUS_ADDRESS'] = ''
            env['NO_DBUS'] = '1'
            
            print("🚀 Starting server with user permissions...")
            self.server_process = subprocess.Popen([
                self.binary_path,
                '--no-sandbox',
                '--headless',
                '--disable-dev-shm-usage',
                '--disable-gpu',
                '--disable-software-rasterizer',
                '--no-first-run',
                '--disable-default-apps',
                '--disable-extensions',
                '--disable-plugins',
                '--disable-sync',
                '--disable-background-timer-throttling',
                '--disable-renderer-backgrounding',
                '--disable-backgrounding-occluded-windows',
                '--disable-ipc-flooding-protection',
                '--disable-features=VizDisplayCompositor,TranslateUI',
                '--disable-logging',
                '--disable-background-mode',
                '--user-data-dir=' + os.path.join(home, '.pinokio-data')
            ], env=env, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            
            # Wait for server
            print("⏳ Waiting for Lightning.ai startup...")
            for i in range(45):  # 45 second timeout
                try:
                    response = requests.get('http://localhost:42000', timeout=3)
                    if response.status_code == 200:
                        print("✅ ⚡ Lightning.ai server running on port 42000!")
                        return True
                except requests.exceptions.RequestException:
                    time.sleep(1)
                    if i % 10 == 0 and i > 0:
                        print(f"⏳ Still starting... ({i+1}/45)")
                        
            print("❌ Server startup timeout - checking process...")
            if self.server_process.poll() is not None:
                stdout, stderr = self.server_process.communicate()
                print(f"❌ Process exited with code {self.server_process.returncode}")
                print(f"Error: {stderr[:500]}")
            return False
            
        except Exception as e:
            print(f"❌ Lightning startup error: {e}")
            return False
    
    def setup_tunnel(self):
        print("🌐 Setting up tunnel for Lightning.ai...")
        
        try:
            # Try Cloudflare tunnel in user space
            cf_path = os.path.join(self.pinokio_path, 'cloudflared')
            if not os.path.exists(cf_path):
                print("📥 Downloading Cloudflare tunnel...")
                subprocess.run(['wget', '-q', '-O', cf_path, 'https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64'])
                os.chmod(cf_path, 0o755)
            
            self.tunnel_process = subprocess.Popen([cf_path, 'tunnel', '--url', 'http://localhost:42000'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            
            print("⏳ Starting tunnel...")
            for _ in range(30):
                line = self.tunnel_process.stdout.readline()
                if 'trycloudflare.com' in line:
                    for part in line.split():
                        if 'trycloudflare.com' in part:
                            self.tunnel_url = part.strip()
                            print(f"✅ ⚡ Lightning tunnel: {self.tunnel_url}")
                            return self.tunnel_url
                time.sleep(1)
            
            print("❌ Tunnel setup failed - continuing with local access")
            return None
            
        except Exception as e:
            print(f"❌ Tunnel error: {e}")
            return None
    
    def cleanup(self):
        if self.server_process:
            self.server_process.terminate()
        if self.tunnel_process:
            self.tunnel_process.terminate()

# LIGHTNING.AI EXECUTION
try:
    print("⚡ STARTING PINOKIO ON LIGHTNING.AI")
    print("=" * 50)
    
    pinokio = PinokioLightning()
    
    pinokio.setup_directory()
    
    if not pinokio.download_pinokio():
        raise Exception("Failed to download Pinokio")
    
    if not pinokio.start_pinokio_server():
        raise Exception("Failed to start Pinokio server")
    
    tunnel_url = pinokio.setup_tunnel()
    
    print("\n⚡ LIGHTNING.AI SUCCESS! ⚡")
    print("=" * 50)
    
    if tunnel_url:
        print(f"🌍 Public URL: {tunnel_url}")
        print("📱 Click the link above to access Pinokio!")
    else:
        print("🖥️ Local URL: http://localhost:42000")
        print("💡 Use port forwarding to access externally")
    
    print("\n📋 What you can do now:")
    print("• Browse the 'Discover' tab for 500+ AI tools")
    print("• Install Stable Diffusion, ComfyUI, etc. with one click")
    print("• Access all tools through the web interface")
    
    _pinokio_instance = pinokio
    _pinokio_url = tunnel_url or "http://localhost:42000"
    
except Exception as e:
    print(f"\n❌ Lightning.ai setup failed: {e}")
    print("\n🔧 Troubleshooting:")
    print("1. Check if you have sufficient resources allocated")
    print("2. Try restarting the Lightning.ai environment")
    print("3. Contact Lightning.ai support if permission errors persist")

In [ ]:
#@title 📊 Status Check
if '_pinokio_instance' in locals():
    try:
        response = requests.get('http://localhost:42000', timeout=2)
        if response.status_code == 200:
            print("✅ ⚡ Pinokio running successfully on Lightning.ai!")
            if '_pinokio_url' in locals():
                print(f"🔗 Access: {_pinokio_url}")
        else:
            print("❌ Server not responding")
    except:
        print("❌ Server not running")
else:
    print("❌ Run the setup cell first")

In [ ]:
#@title 🧹 Cleanup
if '_pinokio_instance' in locals():
    print("🧹 Stopping Pinokio processes...")
    _pinokio_instance.cleanup()
    print("✅ Cleanup complete")
else:
    print("❌ No active instance found")